In [0]:
from google.colab import drive
from google.colab import files

import numpy as np
import pandas as pd


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  output_LGB.csv  user_test.csv    user_train.csv
model_NN.h5    output_NN.csv   user_test_n.csv  user_train_n.csv


In [15]:
%%time

df_NN = pd.read_csv('output_NN.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_LGB = pd.read_csv('output_LGB.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

df_NN.columns = ['NN_pred']
df_LGB.columns = ['LGB_pred']


CPU times: user 2.21 s, sys: 117 ms, total: 2.32 s
Wall time: 2.39 s


In [16]:
df_all = pd.concat((df_NN,df_LGB),axis = 1)
df_all.describe()

,NN_pred,LGB_pred
count,617242.000000,617242.000000
mean,0.247548,0.221480
std,1.439989,1.325335
min,0.000000,0.000000
25%,0.000000,0.016143
50%,0.007620,0.016143
75%,0.026244,0.016143
max,174.396260,131.467144


In [17]:
df_all['predictedLogRevenue'] = (df_all['NN_pred'] + df_all['LGB_pred'])/2
df_all.describe()

,NN_pred,LGB_pred,predictedLogRevenue
count,617242.000000,617242.000000,617242.000000
mean,0.247548,0.221480,0.234514
std,1.439989,1.325335,1.339719
min,0.000000,0.000000,0.000000
25%,0.000000,0.016143,0.008071
50%,0.007620,0.016143,0.012038
75%,0.026244,0.016143,0.022896
max,174.396260,131.467144,143.444353


In [18]:
df_all.corr()

,NN_pred,LGB_pred,predictedLogRevenue
NN_pred,1.000000,0.877485,0.971454
LGB_pred,0.877485,1.000000,0.966212
predictedLogRevenue,0.971454,0.966212,1.000000


In [19]:
%%time

df_all.drop('NN_pred',axis = 1, inplace=True)
df_all.drop('LGB_pred',axis = 1, inplace=True)
df_all[df_all < 0] = 0

CPU times: user 58.3 ms, sys: 2.01 ms, total: 60.3 ms
Wall time: 62.6 ms


In [0]:
df_all.to_csv('output_ens.csv')

In [20]:
df_all.describe()

,predictedLogRevenue
count,617242.000000
mean,0.234514
std,1.339719
min,0.000000
25%,0.008071
50%,0.012038
75%,0.022896
max,143.444353


In [23]:
df_all.head()

,predictedLogRevenue
fullVisitorId,
0000000259678714014,1.026191
0000049363351866189,0.019333
0000053049821714864,0.008071
0000059488412965267,0.019404
0000085840370633780,0.028617


In [24]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 617242 entries, 0000000259678714014 to 9999997304197521748
Data columns (total 1 columns):
predictedLogRevenue    617242 non-null float64
dtypes: float64(1)
memory usage: 9.4+ MB
